In [1]:
from IPython.display import clear_output
!wget https://disk.karelia.pro/djnSMfz/Трек%201.%20Онежское%20озеро.tgz?attachment
import os
try:
    os.mkdir("./data")
except: pass
!tar -xvzf "Трек 1. Онежское озеро.tgz?attachment" -C ./data
clear_output()

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install catboost

Mounted at /content/drive
     |████████████████████████████████| 65.7MB 46kB/s 


In [3]:
path = "./drive/My Drive/Ситилинк Хакатон/"

In [4]:
import os
from torchvision import transforms
import pickle
import pandas as pd
import torch
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
 
class LakeClassifier():
    def __init__(self, models_path):
        self.device = torch.device('cuda' if 
                                   torch.cuda.is_available() else 'cpu')
        self.network = torch.load(os.path.join(models_path, "network.pth"), map_location=torch.device('cpu')).to(self.device)
        self.transform = transforms.ToTensor()
        self.size = (128, 128)
    
    def predict_network(self, path):
        predictions = []
        for filename in tqdm(os.listdir(path)):
            with Image.open(os.path.join(path, filename)) as image:
                image = image.resize(self.size)
            X = self.transform(image)
            X = X.view(1, *X.size()).to(self.device)
            prediction = self.network(X).argmax(dim=1).item()
            predictions.append([filename, prediction])
        return predictions

In [5]:
model_nn = LakeClassifier(path)
predictions = model_nn.predict_network("data")

In [6]:
import pandas as pd

# переводим дату в числа
def date_to_data(frame, holidays=None,
                 drop=False, name="date", suffix=""):
    assert name in frame, "Нужна колонка с датой"
    date = frame[name].dt

    info = pd.concat([date.days_in_month,
                     date.month,
                     date.quarter,
                     date.month % 12 // 3 + 1], axis=1)
    
    info.columns = ["days_in_month", "month",
                    "quarter", "season"]

    temp = pd.concat([date.isocalendar(),
                      info], axis=1)

    temp.columns = temp.columns + suffix
    frame = pd.concat([frame, temp], axis=1)
    if drop:
        return frame.drop([name], axis=1)
    return frame

df = pd.read_csv(path + "joined_last.csv", index_col="Unnamed: 0")
df["date"] = pd.to_datetime(df["name"].apply(lambda x: x[:-4]))
df = date_to_data(df, name="date", drop=False)

In [7]:
arr = []
frame = df[["date", "y"]].values
target = df["y"].to_list()

for i, data in enumerate(frame):
    date, y = data
    if y:
        # arr.append([date, y, 0])
        arr.append(0)
    else:
        idx = i + target[i:].index(1)
        temp_date, temp_y = frame[idx]
        new_date = temp_date - date
        new_date = (new_date.days * 24) + (new_date.seconds // 3600)
        if new_date < 0:
            print(temp_date, date)
            print(temp_date - date)
            print(i, idx)
            break
        # arr.append([date, y, new_date])
        arr.append(new_date)

df["hours_to_freezing"] = arr

In [8]:
predictions_df = pd.DataFrame(predictions, columns=["filename", "prediction"])
frame = df.merge(predictions_df, left_on="name", right_on="filename")

In [9]:
names = ['hour', 'year', 'week', 'day',
        'days_in_month', 'month', 'quarter', 'season',
        'prediction']

In [10]:
from sklearn.model_selection import train_test_split

def train_test_val_split(X, y,
                         test_size=0.3, val_size=0.3,
                         random_state=42, shuffle=True):
    """
    separator for train, test, validation
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        train_size=1 - (test_size + val_size),
                                                        random_state=random_state, shuffle=shuffle)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
                                                    test_size=test_size / (test_size + val_size),
                                                    random_state=random_state, shuffle=shuffle)
    return X_train, y_train, X_test, y_test, X_val, y_val

X = frame[names].astype(np.float32)
y = frame["hours_to_freezing"].astype(np.int32)

X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(X, y,
                                                                      test_size=0.3,
                                                                      val_size=0.3,
                                                                      shuffle=False)

In [11]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=1000,
                          task_type="GPU",
                          loss_function="RMSE",
                          grow_policy='Depthwise')

# model.fit(X_train, y_train)
model.fit(X, y)

Learning rate set to 0.076706
0:	learn: 1983.7634680	total: 9.88ms	remaining: 9.87s
1:	learn: 1844.2136887	total: 18.1ms	remaining: 9.01s
2:	learn: 1718.3993660	total: 26.3ms	remaining: 8.73s
3:	learn: 1604.1419019	total: 38.5ms	remaining: 9.6s
4:	learn: 1507.3716174	total: 55.6ms	remaining: 11.1s
5:	learn: 1406.7327407	total: 63.9ms	remaining: 10.6s
6:	learn: 1322.1766279	total: 72.3ms	remaining: 10.3s
7:	learn: 1241.9342749	total: 86.7ms	remaining: 10.7s
8:	learn: 1162.8313548	total: 102ms	remaining: 11.3s
9:	learn: 1093.0161021	total: 110ms	remaining: 10.9s
10:	learn: 1025.9834665	total: 119ms	remaining: 10.7s
11:	learn: 967.8978446	total: 127ms	remaining: 10.5s
12:	learn: 912.2943889	total: 136ms	remaining: 10.3s
13:	learn: 854.9744023	total: 144ms	remaining: 10.2s
14:	learn: 809.9415311	total: 152ms	remaining: 10s
15:	learn: 761.5191911	total: 161ms	remaining: 9.87s
16:	learn: 718.3631874	total: 169ms	remaining: 9.75s
17:	learn: 676.9586954	total: 177ms	remaining: 9.66s
18:	learn:

In [12]:
from sklearn.metrics import mean_absolute_error

prediction = pd.DataFrame(model.predict(X_test), columns=["pred"])
prediction[prediction["pred"] < 0] = 0
prediction = prediction["pred"].values

mean_absolute_error(y_test, prediction)

7.294161919732195

In [13]:
from sklearn.metrics import mean_absolute_error

prediction = pd.DataFrame(model.predict(X_val), columns=["pred"])
prediction[prediction["pred"] < 0] = 0
prediction = prediction["pred"].values

mean_absolute_error(y_val, prediction)

8.138321640905316

In [14]:
sorted(zip(model.feature_names_, model.feature_importances_),
       reverse=True, key=lambda x: x[1])

[('week', 47.00427028893453),
 ('season', 22.737808627009098),
 ('prediction', 13.656693132500001),
 ('year', 11.119269061540498),
 ('month', 3.8434967151923662),
 ('day', 0.8691087737286247),
 ('days_in_month', 0.5565548070056842),
 ('hour', 0.1471353715259833),
 ('quarter', 0.06566322256321248)]

In [15]:
import pickle
with open(path + "ModernRegressor.pkl", "wb") as f:
    pickle.dump(model, f)